<a href="https://colab.research.google.com/github/lorsange/colab/blob/main/BlenderNerf_InstantNGP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Instant NGP

Check which NVIDIA GPU is being used

In [ ]:
!nvidia-smi

Download and install packages

In [ ]:
!apt update && apt install build-essential git python3-dev python3-pip libopenexr-dev libxi-dev libglfw3-dev libglew-dev libomp-dev libxinerama-dev libxcursor-dev
!pip install --upgrade cmake

Clone the Instant NGP repository

In [ ]:
!git clone --recursive https://github.com/nvlabs/instant-ngp
%cd instant-ngp

Build the project with cmake (and disable GUI, as not supported in python notebooks) : **can take around 10min**

In [ ]:
!cmake . -B build -DNGP_BUILD_WITH_GUI=OFF
!cmake --build build --config RelWithDebInfo -j `nproc`

Install package requirements

In [ ]:
!pip3 install -r requirements.txt

# Upload dataset

You can upload and access your archived *Blender $\times$ NeRF* dataset (compressed in a ZIP file) in 3 different ways. If working with *COLAB*, I suggest uploading your data to Google Drive (*gdrive*) if it exceeds a few megabytes, as the uploading speed is significantly faster

* `google_drive` : upload the ZIP file to gdrive, and edit **<path_to_dataset.zip>** in `gdrive_path` to match the relative path in which the dataset is stored in your gdrive

* `colab_drive` : upload the ZIP file directly to the *COLAB* drive, accessible at the *Files* icon on the left panel. You can then upload your file either by clicking the top left *Upload to session storage* icon, or by just dragging your file to the left *Files* window. Finally, edit **<path_to_dataset.zip>** in `colab_drive` to match the relative path of your just uploaded dataset

* `local_drive` : if you are not working with *COLAB*, but locally on your own machine, you can directly access the dataset by editing **<path_to_dataset.zip>** to the absolute path of your dataset

In [ ]:
upload_type = 'google_drive' #@param ['google_drive', 'colab_drive', 'local_drive'] {allow-input: false}

gdrive_path = '/content/drive/MyDrive/<path_to_dataset.zip>' #@param {type: 'string'}

colab_path = '/content/<path_to_dataset.zip>' #@param {type: 'string'}

local_path = '<path_to_dataset.zip>' #@param {type: 'string'}

Extract the archived dataset

In [ ]:
if upload_type == 'google_drive':
    from google.colab import drive
    drive.mount('/content/drive/')
    dataset_path = gdrive_path

elif upload_type == 'colab_drive':
    dataset_path = colab_path

elif upload_type == 'local_drive':
    dataset_path = local_path

!unzip $dataset_path

# Run NeRF

Set parameters to the ones you defined in the *Blender $\times$ NeRF* add-on

In [ ]:
%cd /content/instant-ngp/

import os
import json

dataset_dir = os.path.basename(dataset_path).split('.zip')[0]
dataset_dir_train = '%s_train' % dataset_dir
dataset_dir_test = '%s_test' % dataset_dir
transforms_test_filepath = '%s/transforms_test.json' % dataset_dir_test

with open(transforms_test_filepath) as file:
    transforms_test = json.loads(file.read())

width = int(transforms_test['w'])
height = int(transforms_test['h'])

test_img_dir = os.path.dirname( transforms_test['frames'][0]['file_path'] )
test_dir = os.path.join(dataset_dir_test, test_img_dir)

**Run NeRF** : the training might take between a few seconds and a few minutes, depending on which GPU was assigned to you on *COLAB*

* Higher *AABB scale* values will increase the training time, as rays will have to cross a larger bounding box area. A larger image resolution will also impact training time

* Per default, I set the number of trainin epochs (or iterations) to 2000. Feel free to lower this number if you believe the model can converge faster

* The predicted test images are stored under `instant-ngp/<dataset_name>_test/`

In [ ]:
!python scripts/run.py --training_data=$dataset_dir_train/ --mode=nerf \
                       --screenshot_transforms=$transforms_test_filepath \
                       --screenshot_w=$width --screenshot_h=$height \
                       --screenshot_dir=$test_dir \
                       --save_snapshot=snapshot.msgpack \
                       --n_steps=2000

Compress predicted test images under `/content/<dataset_name>_test.zip`. The archived file can directly be downloaded from the left *Files* window with a right click

In [ ]:
!zip -r /content/$dataset_dir_test\.zip $dataset_dir_test

Optionally, copy the archived file in your gdrive. If you previously uploaded the dataset via gdrive, the file will be stored under the same path

In [ ]:
if upload_type == 'google_drive':
    save_path = os.path.dirname(gdrive_path)
    !cp /content/$dataset_dir_test\.zip $save_path

else:
    from google.colab import drive
    drive.mount('/content/drive/')
    !cp /content/$dataset_dir_test\.zip /content/drive/MyDrive/